In [ ]:
import random
import plotly.express as plte
import numpy
import pandas

In [ ]:
def check_constraints(x, constrv):
    for con in constrv:
        if con(x) != 0:
            return False
    return True


def bf_sampler(x_from, x_to, iterations, constrs):
    result = []
    for i in range(0, iterations):
        x = []
        for j in range(0, len(x_from)):
            x.append(x_from[j] + random.random()*(x_to[j] - x_from[j]))
        if check_constraints(x, constrs):
            result.append(x)
    return pandas.DataFrame(result)


def range_constraint(i, x_from, x_to):
    return lambda x: abs((x[i] - x_from) - abs(x[i] - x_from)) + abs(
        (x_to - x[i]) - abs(x_to - x[i]))

In [ ]:
x1 = 0
x2 = 1

constraints = [
    range_constraint(i=x1, x_from=10, x_to=100)
]

N = 100000
range_to = [1000]
range_from = [-1000]
dim = 1

res = bf_sampler(x_from=range_from, x_to=range_to,
                 iterations=N, constrs=constraints)


print(f"Filtered/total ratio = {len(res)}/{N} ({len(res)/N*100}%)")
plte.histogram(x=res[0])

Filtered/total ratio = 4496/100000 (4.496%)


In [ ]:
x1 = 0
x2 = 1
dim = 6
x_from = 10
x_to = 100
r_from = -1000
r_to = 1000
N = 1000000

result = []

for i in range(1, dim + 1):
    constraints = []
    for k in range(0, i):
        constraints.append(range_constraint(i=k, x_from=x_from, x_to=x_to))

    range_to = numpy.ones(i)*r_to
    range_from = numpy.ones(i)*r_from

    result.append(bf_sampler(x_from=range_from, x_to=range_to,
                    iterations=N, constrs=constraints))

    print(f"[{i}] Filtered/total ratio = {len(result[-1])}/{N} ({len(result[-1])/N*100}%)")

[1] Filtered/total ratio = 44895/1000000 (4.4895%)
[2] Filtered/total ratio = 2032/1000000 (0.2032%)
[3] Filtered/total ratio = 98/1000000 (0.0098%)
[4] Filtered/total ratio = 4/1000000 (0.00039999999999999996%)
[5] Filtered/total ratio = 0/1000000 (0.0%)


In [ ]:
plte.density_heatmap(result[1], x=0, y=1, range_x=[r_from, r_to], range_y=[r_from, r_to])

In [ ]:
import pymc